<a href="https://colab.research.google.com/github/pinilDissanayaka/sinhala-gpt2-finetune/blob/main/sinhala_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:
df = load_dataset("ihalage/sinhala-finetune-qa-eli5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

sinhala-finetune-qa-eli5.jsonl:   0%|          | 0.00/35.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")


print(f"Available Device : {device}")

Available Device : cuda


In [5]:
df

DatasetDict({
    train: Dataset({
        features: ['q_id', 'subreddit', 'url', 'sinhala_question', 'sinhala_answer', 'english_question', 'english_answer'],
        num_rows: 10000
    })
})

In [6]:
questions=list(df["train"]["sinhala_question"])

answers=list(df["train"]["sinhala_answer"])

In [8]:
from torch.utils.data import Dataset



class TextDataset(Dataset):
    def __init__(self, questions:list, answers:list, tokenizer):
        self.questions = questions
        self.answers = answers
        self.data=list()
        self.tokenizer = tokenizer

        for question, answer in zip(self.questions, self.answers):
            self.data.append("<startofstring> "+question+" <bot>:"+answer+" <endofstring>")

        self.data_tokenized = self.tokenizer(self.data, return_tensors="pt", padding=True, truncation=True)


        self.input_ids = self.data_tokenized["input_ids"]
        self.attention_mask = self.data_tokenized["attention_mask"]




    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [9]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.add_special_tokens({"pad_token": "<pad>", "bos_token": "<startofstring>", "eos_token": "<endofstring>"})

tokenizer.add_tokens(["<bot>:"])


optimizer=Adam(params=model.parameters())

dataset=TextDataset(questions, answers, tokenizer)

data_loader=DataLoader(dataset, batch_size=250, shuffle=True)


def train(model, optimizer, data, epochs):
    model=model.to(device)

    for epoch in tqdm(range(epochs)):
        for input_id, attention_mask in data:

            input_id = input_id.to(device)
            attention_mask = attention_mask.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_id, attention_mask=attention_mask, labels=input_id)

            loss = outputs[0]

            print(f"\n epoch: {epoch} loss: {loss}")

            loss.backward()
            optimizer.step()

    torch.save(model.state_dict(), f"model_epoch_{epochs}.pth")

  return model


def predict(model, text):
    text="<startofstring> "+text+" <bot>:"

    text_tokens=tokenizer(text)

    return model.generale(**text_tokens)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [17]:
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50261, 768)

In [ ]:
model=train(model=model, optimizer=optimizer, data=dataset, epochs=10)